In [1]:
from osgeo import gdal
import sys
import os
import geopandas as gpd
import pandas as pd
import numpy as np

In [40]:
##########################################
### clip raster by study area (vector) ###
##########################################

In [18]:
path_raster = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/march/S2A_20180322/B02_10.jp2"
path_vector = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/temp/study_area/boundry.shp"
path_out = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/march_clip/S2A_20180322/B02_10.jp2"

In [33]:
path_in = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/march/"
path_out = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/march_clip/"

In [34]:
dirs_rast = sorted(os.listdir(path_in))

for dir_rast in dirs_rast[2:]:
    print('*********************')
    print(dir_rast)
    print(sorted(os.listdir(path_in+dir_rast))[:10])
    #poi = extract_values(poi, path_raster+dir_rast+'/')

*********************
S2A_20180322
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180421
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180513
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180617
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180702
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180720
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.j

In [35]:
# load vector - study area boundries
vect = gpd.read_file(path_vector)
# get extend
extend = vect.loc[0,'geometry'].bounds

In [36]:
extend

(602197.2084267415, 5330943.329159929, 647517.2084267415, 5373373.329159929)

In [39]:
dir_dates = sorted(os.listdir(path_in))

# loop over dates
for dir_date in dir_dates[2:]:
    print('*********************')
    print(dir_date)
    dir_bands = sorted(os.listdir(path+dir_date))[:10]
    
    # loop over bands
    for dir_band in dir_bands:
        rast_in = "{}{}/{}".format(path_in,dir_date,dir_band)
        rast_out = "{}{}/{}".format(path_out,dir_date,dir_band)
        
        # construct bash command
        bash_command = "gdal_translate -projwin {} {} {} {} -a_nodata 0.0 -of GTiff {} {}".format(extend[0], extend[3], extend[2], extend[1], rast_in, rast_out)
        # execute command
        os.system(bash_command)

*********************
S2A_20180322
*********************
S2A_20180421
*********************
S2A_20180513
*********************
S2A_20180617
*********************
S2A_20180702
*********************
S2A_20180720
*********************
S2A_20180809
*********************
S2A_20180821
*********************
S2A_20180920


In [ ]:
###############################

In [16]:
# create nupy template
x = np.zeros([4244,4533], dtype='uint8')

In [18]:
x.shape

(4244, 4533)

In [ ]:
Guten Abend Christian,
ich bin an den RAM-Modulen interessiert. Sind diese noch verfügbar? Wie würde die Geldüberweisung funktionieren bei Versand?

In [ ]:
boundries = gpd.

In [ ]:
clip_by_grid(path_vector, path_raster, )

In [7]:
def clip_by_grid(path_in_grid, path_in_raster, path_out_dir, data_type='ortho'):
    '''
    clip raster by grid

    input:
        path_in_grid (string) - path to vector (.shp) file containing polygons
        path_in_raster (string) - path to raster (.tif) that needs to be cutted
        path_out_dir (string) -
        data_type (string) - name of type (exp. 'ortho', 'dsm', 'dtm', 'slope')
    '''

    # load vector file
    grid = geopandas.read_file(path_in_grid)

    if (data_type == 'ortho') or (data_type == 'prediction'):
        sufix = ''
    else:
        sufix = '_1m'

    # variables for keeping track of the progress
    size = grid.shape[0]
    percentage = 0

    # loop over all the polygons
    for i in range(size):
        # keep track of the progress
        if i % (size // 10) == 0:
            print('{}%'.format(percentage))
            percentage += 10
        # get extend
        extend = grid.loc[i,'geometry'].bounds
        # set paths
        path_out = "{}{}{}/tile_{}_{}.tif".format(path_out_dir, data_type, sufix, data_type, grid.loc[i,'id'])

        # construct bash command
        bash_command = "gdal_translate -projwin {} {} {} {} -a_nodata 0.0 -of GTiff {} {}".format(extend[0], extend[3], extend[2], extend[1], path_in_raster, path_out)
        # execute command
        os.system(bash_command)

    print("finished cilpping")

In [13]:
19238052/4533

4244.0

In [1]:

# Extent
extend = [605726.1051020468585193, 5332748.8710450790822506, 646739.8407182103255764, 5370219.1156320208683610]

In [3]:
extend[2]-extend[0]

41013.73561616347

In [4]:
extend[3]-extend[1]

37470.244586941786

In [ ]:
def tif2array(self, file_path, dtype=np.uint8):
    """
    read GeoTiff and convert to numpy.ndarray.
    inputs:
        file_path (str) : file path of the input GeoTiff file
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive
    """
    dataset = gdal.Open(file_path, gdal.GA_ReadOnly)

    if dataset is None:
        return None

    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=dtype)

    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        image[:, :, b] = band.ReadAsArray()#buf_type=gdalconst.GDT_Byte)

    #image = image[2:-2,2:-2,:]

    return image

In [ ]:
# create numpy array

In [ ]:
# classify

In [ ]:
# create image